# Breakhis-400x Dataset

In [26]:
print("Welcome")

Welcome


# Final DNBCD Model (Custom CNN + DenseNet + Transfer Learning)

In [9]:
import os
import time
import numpy as np
import cv2  
import tensorflow as tf
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import DenseNet121
from sklearn.utils.class_weight import compute_class_weight

# Define paths
dataset_path = '/kaggle/input/breakhis-400x/Breakhis-400x'
base_dir = '/kaggle/working/split_data'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

# Ensure clean split directories
for dir_path in [train_dir, val_dir, test_dir]:
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
    os.makedirs(dir_path)

# Split dataset into 70% train, 20% test, 10% validation
all_images = []
for class_name in os.listdir(dataset_path):
    class_dir = os.path.join(dataset_path, class_name)
    images = [os.path.join(class_dir, img) for img in os.listdir(class_dir)]
    all_images.extend([(img, class_name) for img in images])

test_split = 0.2
val_split = 0.1
train_images, temp_images = train_test_split(
    all_images,
    test_size=(test_split + val_split),
    stratify=[label for _, label in all_images],
    random_state=42
)
val_images, test_images = train_test_split(
    temp_images,
    test_size=(test_split / (test_split + val_split)),
    stratify=[label for _, label in temp_images],
    random_state=42
)

# Copy files into respective directories without preprocessing
def copy_images(images, target_dir):
    for img_path, label in images:
        label_dir = os.path.join(target_dir, label)
        os.makedirs(label_dir, exist_ok=True)
        shutil.copy(img_path, label_dir)

copy_images(train_images, train_dir)
copy_images(val_images, val_dir)
copy_images(test_images, test_dir)

# Parameters
input_shape = (128, 128, 3)
batch_size = 32
epochs = 20

# Create data generators for binary classification
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='binary'
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

# Print the number of samples in each split
print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {validation_generator.samples}")
print(f"Test samples: {test_generator.samples}")

# Compute class weights
train_labels = train_generator.classes
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)
class_weights = dict(enumerate(class_weights))
print("Class Weights:", class_weights)

# Load pre-trained DenseNet121 model + higher-level layers
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze the base model
base_model.trainable = False

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)

# Create the complete model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Start profiling
tf.keras.backend.clear_session()
tf.profiler.experimental.start('logdir/profile')

start_time = time.time()
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    class_weight=class_weights
)
print("Initial training time: {:.2f} seconds".format(time.time() - start_time))

# Stop profiling
tf.profiler.experimental.stop()

# Unfreeze the base model for fine-tuning
base_model.trainable = True
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Fine-tune the model with class weights
start_time = time.time()
history_fine = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    class_weight=class_weights
)
print("Fine-tuning training time: {:.2f} seconds".format(time.time() - start_time))

# Evaluate the model on the test set
start_time = time.time()
loss, accuracy = model.evaluate(test_generator)
print("Test evaluation time: {:.2f} seconds".format(time.time() - start_time))

Found 1273 images belonging to 2 classes.
Found 182 images belonging to 2 classes.
Found 365 images belonging to 2 classes.
Training samples: 1273
Validation samples: 182
Test samples: 365
Class Weights: {0: 1.548661800486618, 1: 0.738399071925754}
Epoch 1/20


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


40/40 ━━━━━━━━━━━━━━━━━━━━ 47s 613ms/step - accuracy: 0.5810 - loss: 0.8738 - val_accuracy: 0.7308 - val_loss: 0.5869
Epoch 2/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 153ms/step - accuracy: 0.6151 - loss: 0.6572 - val_accuracy: 0.7692 - val_loss: 0.5484
Epoch 3/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 151ms/step - accuracy: 0.6915 - loss: 0.5700 - val_accuracy: 0.8352 - val_loss: 0.4990
Epoch 4/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 152ms/step - accuracy: 0.6846 - loss: 0.5764 - val_accuracy: 0.7747 - val_loss: 0.5046
Epoch 5/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 151ms/step - accuracy: 0.7456 - loss: 0.5273 - val_accuracy: 0.7253 - val_loss: 0.5273
Epoch 6/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 153ms/step - accuracy: 0.7743 - loss: 0.4579 - val_accuracy: 0.7088 - val_loss: 0.5098
Epoch 7/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 152ms/step - accuracy: 0.7736 - loss: 0.4680 - val_accuracy: 0.8352 - val_loss: 0.4286
Epoch 8/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 152ms/step - accuracy: 0.7832 - loss: 0.4674 - val_accuracy: 0.8132 - val

# Mobilenet+transfer

In [10]:
import os
import numpy as np
import tensorflow as tf
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNet
from sklearn.utils.class_weight import compute_class_weight
import time

# Define paths
dataset_path = '/kaggle/input/breakhis-400x/Breakhis-400x'
base_dir = '/kaggle/working/split_data'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

# Ensure clean split directories
for dir_path in [train_dir, val_dir, test_dir]:
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
    os.makedirs(dir_path)

# Split dataset into 70% train, 20% test, 10% validation
all_images = []
for class_name in os.listdir(dataset_path):
    class_dir = os.path.join(dataset_path, class_name)
    images = [os.path.join(class_dir, img) for img in os.listdir(class_dir)]
    all_images.extend([(img, class_name) for img in images])

test_split = 0.2
val_split = 0.1
train_images, temp_images = train_test_split(
    all_images,
    test_size=(test_split + val_split),
    stratify=[label for _, label in all_images],
    random_state=42
)
val_images, test_images = train_test_split(
    temp_images,
    test_size=(test_split / (test_split + val_split)),
    stratify=[label for _, label in temp_images],
    random_state=42
)

# Copy files into respective directories without preprocessing
def copy_images(images, target_dir):
    for img_path, label in images:
        label_dir = os.path.join(target_dir, label)
        os.makedirs(label_dir, exist_ok=True)
        shutil.copy(img_path, label_dir)

copy_images(train_images, train_dir)
copy_images(val_images, val_dir)
copy_images(test_images, test_dir)

# Parameters
input_shape = (128, 128, 3)
batch_size = 32
epochs = 20

# Create data generators for binary classification
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='binary'
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

# Print the number of samples in each split
print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {validation_generator.samples}")
print(f"Test samples: {test_generator.samples}")

# Compute class weights
train_labels = train_generator.classes
class_weights_array = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)
class_weights = dict(enumerate(class_weights_array))
print("Class Weights:", class_weights)

# Load pre-trained MobileNet model + higher-level layers
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze the base model
base_model.trainable = False

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)

# Create the complete model
mobilenet_model = Model(inputs=base_model.input, outputs=output)

# Compile the model
mobilenet_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Start profiling for initial training
start_time = time.time()
mobilenet_history_initial = mobilenet_model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    class_weight=class_weights
)
print("Initial training time: {:.2f} seconds".format(time.time() - start_time))

# Unfreeze the base model for fine-tuning
base_model.trainable = True
mobilenet_model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Start profiling for fine-tuning
start_time = time.time()
mobilenet_history_fine = mobilenet_model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    class_weight=class_weights
)
print("Fine-tuning training time: {:.2f} seconds".format(time.time() - start_time))

# Evaluate the model on the test set
start_time = time.time()
loss, accuracy = mobilenet_model.evaluate(test_generator)
print("Test evaluation time: {:.2f} seconds".format(time.time() - start_time))

Found 1273 images belonging to 2 classes.
Found 182 images belonging to 2 classes.
Found 365 images belonging to 2 classes.
Training samples: 1273
Validation samples: 182
Test samples: 365
Class Weights: {0: 1.548661800486618, 1: 0.738399071925754}
Epoch 1/20


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


40/40 ━━━━━━━━━━━━━━━━━━━━ 23s 289ms/step - accuracy: 0.5467 - loss: 0.9591 - val_accuracy: 0.7802 - val_loss: 0.5379
Epoch 2/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 145ms/step - accuracy: 0.6478 - loss: 0.6576 - val_accuracy: 0.7253 - val_loss: 0.5639
Epoch 3/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 145ms/step - accuracy: 0.7003 - loss: 0.5895 - val_accuracy: 0.7857 - val_loss: 0.4818
Epoch 4/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 145ms/step - accuracy: 0.7511 - loss: 0.5283 - val_accuracy: 0.8132 - val_loss: 0.4663
Epoch 5/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 148ms/step - accuracy: 0.7358 - loss: 0.5232 - val_accuracy: 0.8022 - val_loss: 0.4646
Epoch 6/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 144ms/step - accuracy: 0.7635 - loss: 0.5155 - val_accuracy: 0.8242 - val_loss: 0.4310
Epoch 7/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 147ms/step - accuracy: 0.7875 - loss: 0.4410 - val_accuracy: 0.8407 - val_loss: 0.3927
Epoch 8/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 145ms/step - accuracy: 0.7984 - loss: 0.4411 - val_accuracy: 0.8407 - val

# Resnet50+transfer

In [11]:
import os
import numpy as np
import cv2  # OpenCV for general use (not CLAHE)
import tensorflow as tf
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from sklearn.utils.class_weight import compute_class_weight
import time

# Optional: For reproducibility
import random
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

# Define paths
dataset_path = '/kaggle/input/breakhis-400x/Breakhis-400x'
base_dir = '/kaggle/working/split_data'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

# Ensure clean split directories
for dir_path in [train_dir, val_dir, test_dir]:
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
    os.makedirs(dir_path)

# Split dataset into 70% train, 20% test, 10% validation
all_images = []
for class_name in os.listdir(dataset_path):
    class_dir = os.path.join(dataset_path, class_name)
    images = [os.path.join(class_dir, img) for img in os.listdir(class_dir)]
    all_images.extend([(img, class_name) for img in images])

test_split = 0.2
val_split = 0.1
train_images, temp_images = train_test_split(
    all_images,
    test_size=(test_split + val_split),
    stratify=[label for _, label in all_images],
    random_state=42
)
val_images, test_images = train_test_split(
    temp_images,
    test_size=(test_split / (test_split + val_split)),
    stratify=[label for _, label in temp_images],
    random_state=42
)

# Copy files into respective directories without preprocessing
def copy_images(images, target_dir):
    for img_path, label in images:
        label_dir = os.path.join(target_dir, label)
        os.makedirs(label_dir, exist_ok=True)
        shutil.copy(img_path, label_dir)  # Copy the original image without preprocessing

copy_images(train_images, train_dir)
copy_images(val_images, val_dir)
copy_images(test_images, test_dir)

# Parameters
input_shape = (128, 128, 3)
batch_size = 32
epochs = 20

# Create data generators for binary classification
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='binary'  # Binary classification
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='binary'  # Binary classification
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='binary',  # Binary classification
    shuffle=False
)

# Print the number of samples in each split
print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {validation_generator.samples}")
print(f"Test samples: {test_generator.samples}")

# Compute class weights
train_labels = train_generator.classes  # Class indices for training samples
class_weights_array = compute_class_weight(
    class_weight='balanced',  # Balance classes based on their frequency
    classes=np.unique(train_labels),
    y=train_labels
)
class_weights = dict(enumerate(class_weights_array))

# Print the calculated class weights
print("Class Weights:", class_weights)

# Load pre-trained ResNet50 model + higher-level layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze the base model
base_model.trainable = False

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)  # Binary classification

# Create the complete model
resnet50_model = Model(inputs=base_model.input, outputs=output)

# Compile the model
resnet50_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Start profiling for initial training
start_time = time.time()
resnet50_history_initial = resnet50_model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    class_weight=class_weights  # Use calculated class weights
)
print("Initial training time: {:.2f} seconds".format(time.time() - start_time))

# Unfreeze the base model for fine-tuning
base_model.trainable = True
resnet50_model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])  # Reduce learning rate for fine-tuning

# Start profiling for fine-tuning
start_time = time.time()
resnet50_history_fine = resnet50_model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    class_weight=class_weights
)
print("Fine-tuning training time: {:.2f} seconds".format(time.time() - start_time))

# Evaluate the model on the test set
start_time = time.time()
loss, accuracy = resnet50_model.evaluate(test_generator)
print("Test evaluation time: {:.2f} seconds".format(time.time() - start_time))

Found 1273 images belonging to 2 classes.
Found 182 images belonging to 2 classes.
Found 365 images belonging to 2 classes.
Training samples: 1273
Validation samples: 182
Test samples: 365
Class Weights: {0: 1.548661800486618, 1: 0.738399071925754}
Epoch 1/20


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


40/40 ━━━━━━━━━━━━━━━━━━━━ 27s 375ms/step - accuracy: 0.4884 - loss: 0.8163 - val_accuracy: 0.3626 - val_loss: 0.7000
Epoch 2/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 149ms/step - accuracy: 0.4526 - loss: 0.7211 - val_accuracy: 0.6758 - val_loss: 0.6818
Epoch 3/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 150ms/step - accuracy: 0.5077 - loss: 0.7001 - val_accuracy: 0.6758 - val_loss: 0.6751
Epoch 4/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 147ms/step - accuracy: 0.4753 - loss: 0.7141 - val_accuracy: 0.6758 - val_loss: 0.6775
Epoch 5/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 147ms/step - accuracy: 0.5590 - loss: 0.6924 - val_accuracy: 0.6758 - val_loss: 0.6831
Epoch 6/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 149ms/step - accuracy: 0.5059 - loss: 0.6997 - val_accuracy: 0.6758 - val_loss: 0.6850
Epoch 7/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 151ms/step - accuracy: 0.5632 - loss: 0.6840 - val_accuracy: 0.4451 - val_loss: 0.6931
Epoch 8/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 148ms/step - accuracy: 0.5249 - loss: 0.6889 - val_accuracy: 0.3297 - val

# Vgg19+transfer

In [12]:
import os
import numpy as np
import cv2  # OpenCV for general use (not CLAHE)
import tensorflow as tf
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from sklearn.utils.class_weight import compute_class_weight
import time

# Optional: For reproducibility
import random
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

# Define paths
dataset_path = '/kaggle/input/breakhis-400x/Breakhis-400x'
base_dir = '/kaggle/working/split_data'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

# Ensure clean split directories
for dir_path in [train_dir, val_dir, test_dir]:
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
    os.makedirs(dir_path)

# Split dataset into 70% train, 20% test, 10% validation
all_images = []
for class_name in os.listdir(dataset_path):
    class_dir = os.path.join(dataset_path, class_name)
    images = [os.path.join(class_dir, img) for img in os.listdir(class_dir)]
    all_images.extend([(img, class_name) for img in images])

test_split = 0.2
val_split = 0.1
train_images, temp_images = train_test_split(
    all_images,
    test_size=(test_split + val_split),
    stratify=[label for _, label in all_images],
    random_state=42
)
val_images, test_images = train_test_split(
    temp_images,
    test_size=(test_split / (test_split + val_split)),
    stratify=[label for _, label in temp_images],
    random_state=42
)

# Copy files into respective directories without preprocessing
def copy_images(images, target_dir):
    for img_path, label in images:
        label_dir = os.path.join(target_dir, label)
        os.makedirs(label_dir, exist_ok=True)
        shutil.copy(img_path, label_dir)  # Copy the original image without preprocessing

copy_images(train_images, train_dir)
copy_images(val_images, val_dir)
copy_images(test_images, test_dir)

# Parameters
input_shape = (128, 128, 3)
batch_size = 32
epochs = 20

# Create data generators for binary classification
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # VGG19-specific preprocessing
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='binary'  # Binary classification
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='binary'  # Binary classification
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='binary',  # Binary classification
    shuffle=False
)

# Print the number of samples in each split
print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {validation_generator.samples}")
print(f"Test samples: {test_generator.samples}")

# Compute class weights
train_labels = train_generator.classes  # Class indices for training samples
class_weights_array = compute_class_weight(
    class_weight='balanced',  # Balance classes based on their frequency
    classes=np.unique(train_labels),
    y=train_labels
)
class_weights = dict(enumerate(class_weights_array))

# Print the calculated class weights
print("Class Weights:", class_weights)

# Load pre-trained VGG19 model + higher-level layers
base_model = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze the base model
base_model.trainable = False

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)  # Binary classification

# Create the complete model
vgg19_model = Model(inputs=base_model.input, outputs=output)

# Compile the model
vgg19_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Start profiling for initial training
start_time = time.time()
vgg19_history_initial = vgg19_model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    class_weight=class_weights  # Use calculated class weights
)
print("Initial training time: {:.2f} seconds".format(time.time() - start_time))

# Unfreeze the base model for fine-tuning
base_model.trainable = True
vgg19_model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])  # Reduce learning rate for fine-tuning

# Start profiling for fine-tuning
start_time = time.time()
vgg19_history_fine = vgg19_model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    class_weight=class_weights
)
print("Fine-tuning training time: {:.2f} seconds".format(time.time() - start_time))

# Evaluate the model on the test set
start_time = time.time()
loss, accuracy = vgg19_model.evaluate(test_generator)
print("Test evaluation time: {:.2f} seconds".format(time.time() - start_time))

Found 1273 images belonging to 2 classes.
Found 182 images belonging to 2 classes.
Found 365 images belonging to 2 classes.
Training samples: 1273
Validation samples: 182
Test samples: 365
Class Weights: {0: 1.548661800486618, 1: 0.738399071925754}
80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/20


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


40/40 ━━━━━━━━━━━━━━━━━━━━ 24s 377ms/step - accuracy: 0.6274 - loss: 2.2554 - val_accuracy: 0.7747 - val_loss: 0.4400
Epoch 2/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 149ms/step - accuracy: 0.6718 - loss: 1.0267 - val_accuracy: 0.7967 - val_loss: 0.4257
Epoch 3/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 151ms/step - accuracy: 0.7366 - loss: 0.6772 - val_accuracy: 0.8187 - val_loss: 0.4377
Epoch 4/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 155ms/step - accuracy: 0.7331 - loss: 0.5968 - val_accuracy: 0.8462 - val_loss: 0.4593
Epoch 5/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 151ms/step - accuracy: 0.7292 - loss: 0.5976 - val_accuracy: 0.8681 - val_loss: 0.4286
Epoch 6/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 152ms/step - accuracy: 0.7429 - loss: 0.5755 - val_accuracy: 0.8571 - val_loss: 0.3998
Epoch 7/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 148ms/step - accuracy: 0.7842 - loss: 0.4884 - val_accuracy: 0.7857 - val_loss: 0.4881
Epoch 8/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 149ms/step - accuracy: 0.7828 - loss: 0.5221 - val_accuracy: 0.8352 - val

# DenseNet121

In [13]:
import os
import numpy as np
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import DenseNet121
from sklearn.utils.class_weight import compute_class_weight
import time

# Define paths
dataset_path = '/kaggle/input/breakhis-400x/Breakhis-400x'
base_dir = '/kaggle/working/split_data'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

# Ensure clean split directories
for dir_path in [train_dir, val_dir, test_dir]:
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
    os.makedirs(dir_path)

# Split dataset into 70% train, 20% test, 10% validation
all_images = []
for class_name in os.listdir(dataset_path):
    class_dir = os.path.join(dataset_path, class_name)
    images = [os.path.join(class_dir, img) for img in os.listdir(class_dir)]
    all_images.extend([(img, class_name) for img in images])

test_split = 0.2
val_split = 0.1
train_images, temp_images = train_test_split(
    all_images,
    test_size=(test_split + val_split),
    stratify=[label for _, label in all_images],
    random_state=42
)
val_images, test_images = train_test_split(
    temp_images,
    test_size=(test_split / (test_split + val_split)),
    stratify=[label for _, label in temp_images],
    random_state=42
)

# Copy files into respective directories without preprocessing
def copy_images(images, target_dir):
    for img_path, label in images:
        label_dir = os.path.join(target_dir, label)
        os.makedirs(label_dir, exist_ok=True)
        shutil.copy(img_path, label_dir)

copy_images(train_images, train_dir)
copy_images(val_images, val_dir)
copy_images(test_images, test_dir)

# Parameters
input_shape = (128, 128, 3)
batch_size = 32
epochs = 20

# Create data generators for binary classification
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='binary'
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

# Print the number of samples in each split
print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {validation_generator.samples}")
print(f"Test samples: {test_generator.samples}")

# Compute class weights
train_labels = train_generator.classes
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)
class_weights = dict(enumerate(class_weights))

# Print the calculated class weights
print("Class Weights:", class_weights)

# Load pre-trained DenseNet121 model + higher-level layers
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze the base model
base_model.trainable = False

# Add Global Average Pooling and output layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
output = Dense(1, activation='sigmoid')(x)  # Binary classification output

# Create the complete model
model_densenet = Model(inputs=base_model.input, outputs=output)

# Compile the model
model_densenet.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Start profiling for training
start_time = time.time()
history_densenet = model_densenet.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    class_weight=class_weights
)
print("Training time: {:.2f} seconds".format(time.time() - start_time))

# Evaluate the model on the test set
start_time = time.time()
loss, accuracy = model_densenet.evaluate(test_generator)
print("Test evaluation time: {:.2f} seconds".format(time.time() - start_time))

Found 1273 images belonging to 2 classes.
Found 182 images belonging to 2 classes.
Found 365 images belonging to 2 classes.
Training samples: 1273
Validation samples: 182
Test samples: 365
Class Weights: {0: 1.548661800486618, 1: 0.738399071925754}
Epoch 1/20


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


40/40 ━━━━━━━━━━━━━━━━━━━━ 40s 557ms/step - accuracy: 0.4976 - loss: 0.8280 - val_accuracy: 0.4011 - val_loss: 0.8081
Epoch 2/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 152ms/step - accuracy: 0.4722 - loss: 0.7923 - val_accuracy: 0.4505 - val_loss: 0.7837
Epoch 3/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 154ms/step - accuracy: 0.4908 - loss: 0.7533 - val_accuracy: 0.4670 - val_loss: 0.7662
Epoch 4/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 148ms/step - accuracy: 0.5108 - loss: 0.7501 - val_accuracy: 0.4451 - val_loss: 0.7641
Epoch 5/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 147ms/step - accuracy: 0.5296 - loss: 0.7435 - val_accuracy: 0.4835 - val_loss: 0.7372
Epoch 6/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 148ms/step - accuracy: 0.5470 - loss: 0.7192 - val_accuracy: 0.4835 - val_loss: 0.7233
Epoch 7/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 153ms/step - accuracy: 0.5771 - loss: 0.7047 - val_accuracy: 0.4890 - val_loss: 0.7093
Epoch 8/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 147ms/step - accuracy: 0.6144 - loss: 0.6785 - val_accuracy: 0.4780 - val

# Mobilenet

In [14]:
import os
import numpy as np
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNet
from sklearn.utils.class_weight import compute_class_weight
import time

# Define paths
dataset_path = '/kaggle/input/breakhis-400x/Breakhis-400x'
base_dir = '/kaggle/working/split_data'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

# Ensure clean split directories
for dir_path in [train_dir, val_dir, test_dir]:
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
    os.makedirs(dir_path)

# Split dataset into 70% train, 20% test, 10% validation
all_images = []
for class_name in os.listdir(dataset_path):
    class_dir = os.path.join(dataset_path, class_name)
    images = [os.path.join(class_dir, img) for img in os.listdir(class_dir)]
    all_images.extend([(img, class_name) for img in images])

test_split = 0.2
val_split = 0.1
train_images, temp_images = train_test_split(
    all_images,
    test_size=(test_split + val_split),
    stratify=[label for _, label in all_images],
    random_state=42
)
val_images, test_images = train_test_split(
    temp_images,
    test_size=(test_split / (test_split + val_split)),
    stratify=[label for _, label in temp_images],
    random_state=42
)

# Copy files into respective directories without preprocessing
def copy_images(images, target_dir):
    for img_path, label in images:
        label_dir = os.path.join(target_dir, label)
        os.makedirs(label_dir, exist_ok=True)
        shutil.copy(img_path, label_dir)

copy_images(train_images, train_dir)
copy_images(val_images, val_dir)
copy_images(test_images, test_dir)

# Parameters
input_shape = (128, 128, 3)
batch_size = 32
epochs = 20

# Create data generators for binary classification
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='binary'
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

# Print the number of samples in each split
print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {validation_generator.samples}")
print(f"Test samples: {test_generator.samples}")

# Compute class weights
train_labels = train_generator.classes
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)
class_weights = dict(enumerate(class_weights))

# Print the calculated class weights
print("Class Weights:", class_weights)

# Load pre-trained MobileNet model + higher-level layers
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze the base model
base_model.trainable = False

# Add Global Average Pooling and output layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
output = Dense(1, activation='sigmoid')(x)  # Binary classification output

# Create the complete model
model_mobilenet = Model(inputs=base_model.input, outputs=output)

# Compile the model
model_mobilenet.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Start profiling for training
start_time = time.time()
history_mobilenet = model_mobilenet.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    class_weight=class_weights
)
print("Training time: {:.2f} seconds".format(time.time() - start_time))

# Evaluate the model on the test set
start_time = time.time()
loss, accuracy = model_mobilenet.evaluate(test_generator)
print("Test evaluation time: {:.2f} seconds".format(time.time() - start_time))

Found 1273 images belonging to 2 classes.
Found 182 images belonging to 2 classes.
Found 365 images belonging to 2 classes.
Training samples: 1273
Validation samples: 182
Test samples: 365
Class Weights: {0: 1.548661800486618, 1: 0.738399071925754}
Epoch 1/20


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


40/40 ━━━━━━━━━━━━━━━━━━━━ 13s 215ms/step - accuracy: 0.5029 - loss: 0.7631 - val_accuracy: 0.4341 - val_loss: 0.8511
Epoch 2/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 143ms/step - accuracy: 0.5817 - loss: 0.7096 - val_accuracy: 0.4451 - val_loss: 0.8293
Epoch 3/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 145ms/step - accuracy: 0.5713 - loss: 0.6886 - val_accuracy: 0.4560 - val_loss: 0.8124
Epoch 4/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 146ms/step - accuracy: 0.6020 - loss: 0.6895 - val_accuracy: 0.4890 - val_loss: 0.7807
Epoch 5/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 144ms/step - accuracy: 0.6361 - loss: 0.6391 - val_accuracy: 0.5110 - val_loss: 0.7577
Epoch 6/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 145ms/step - accuracy: 0.6602 - loss: 0.6173 - val_accuracy: 0.5385 - val_loss: 0.7321
Epoch 7/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 144ms/step - accuracy: 0.6321 - loss: 0.6365 - val_accuracy: 0.5549 - val_loss: 0.7168
Epoch 8/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 145ms/step - accuracy: 0.6876 - loss: 0.5873 - val_accuracy: 0.5604 - val

# Resnet50

In [15]:
import os
import numpy as np
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from sklearn.utils.class_weight import compute_class_weight
import time

# Define paths
dataset_path = '/kaggle/input/breakhis-400x/Breakhis-400x'
base_dir = '/kaggle/working/split_data'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

# Ensure clean split directories
for dir_path in [train_dir, val_dir, test_dir]:
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
    os.makedirs(dir_path)

# Split dataset into 70% train, 20% test, 10% validation
all_images = []
for class_name in os.listdir(dataset_path):
    class_dir = os.path.join(dataset_path, class_name)
    images = [os.path.join(class_dir, img) for img in os.listdir(class_dir)]
    all_images.extend([(img, class_name) for img in images])

test_split = 0.2
val_split = 0.1
train_images, temp_images = train_test_split(
    all_images,
    test_size=(test_split + val_split),
    stratify=[label for _, label in all_images],
    random_state=42
)
val_images, test_images = train_test_split(
    temp_images,
    test_size=(test_split / (test_split + val_split)),
    stratify=[label for _, label in temp_images],
    random_state=42
)

# Copy files into respective directories without preprocessing
def copy_images(images, target_dir):
    for img_path, label in images:
        label_dir = os.path.join(target_dir, label)
        os.makedirs(label_dir, exist_ok=True)
        shutil.copy(img_path, label_dir)

copy_images(train_images, train_dir)
copy_images(val_images, val_dir)
copy_images(test_images, test_dir)

# Parameters
input_shape = (128, 128, 3)
batch_size = 32
epochs = 20

# Create data generators for binary classification
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='binary'
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

# Print the number of samples in each split
print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {validation_generator.samples}")
print(f"Test samples: {test_generator.samples}")

# Compute class weights
train_labels = train_generator.classes
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)
class_weights = dict(enumerate(class_weights))

# Print the calculated class weights
print("Class Weights:", class_weights)

# Load pre-trained ResNet50 model + higher-level layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze the base model
base_model.trainable = False

# Add Global Average Pooling and output layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
output = Dense(1, activation='sigmoid')(x)  # Binary classification output

# Create the complete model
model_resnet = Model(inputs=base_model.input, outputs=output)

# Compile the model
model_resnet.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Start profiling for training
start_time = time.time()
history_resnet = model_resnet.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    class_weight=class_weights
)
print("Training time: {:.2f} seconds".format(time.time() - start_time))

# Evaluate the model on the test set
start_time = time.time()
loss, accuracy = model_resnet.evaluate(test_generator)
print("Test evaluation time: {:.2f} seconds".format(time.time() - start_time))

Found 1273 images belonging to 2 classes.
Found 182 images belonging to 2 classes.
Found 365 images belonging to 2 classes.
Training samples: 1273
Validation samples: 182
Test samples: 365
Class Weights: {0: 1.548661800486618, 1: 0.738399071925754}
Epoch 1/20


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


40/40 ━━━━━━━━━━━━━━━━━━━━ 25s 289ms/step - accuracy: 0.3442 - loss: 0.7059 - val_accuracy: 0.3352 - val_loss: 0.7025
Epoch 2/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 145ms/step - accuracy: 0.3311 - loss: 0.6992 - val_accuracy: 0.3571 - val_loss: 0.6969
Epoch 3/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 144ms/step - accuracy: 0.4617 - loss: 0.6837 - val_accuracy: 0.5714 - val_loss: 0.6910
Epoch 4/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 146ms/step - accuracy: 0.5389 - loss: 0.6878 - val_accuracy: 0.5440 - val_loss: 0.6931
Epoch 5/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 143ms/step - accuracy: 0.4029 - loss: 0.7021 - val_accuracy: 0.3791 - val_loss: 0.6955
Epoch 6/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 145ms/step - accuracy: 0.4469 - loss: 0.6853 - val_accuracy: 0.5769 - val_loss: 0.6921
Epoch 7/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 148ms/step - accuracy: 0.5410 - loss: 0.6868 - val_accuracy: 0.6044 - val_loss: 0.6917
Epoch 8/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 146ms/step - accuracy: 0.4985 - loss: 0.6916 - val_accuracy: 0.6044 - val

# Vgg19

In [16]:
import os
import numpy as np
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG19
from sklearn.utils.class_weight import compute_class_weight
import time

# Define paths
dataset_path = '/kaggle/input/breakhis-400x/Breakhis-400x'
base_dir = '/kaggle/working/split_data'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

# Ensure clean split directories
for dir_path in [train_dir, val_dir, test_dir]:
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
    os.makedirs(dir_path)

# Split dataset into 70% train, 20% test, 10% validation
all_images = []
for class_name in os.listdir(dataset_path):
    class_dir = os.path.join(dataset_path, class_name)
    images = [os.path.join(class_dir, img) for img in os.listdir(class_dir)]
    all_images.extend([(img, class_name) for img in images])

test_split = 0.2
val_split = 0.1
train_images, temp_images = train_test_split(
    all_images,
    test_size=(test_split + val_split),
    stratify=[label for _, label in all_images],
    random_state=42
)
val_images, test_images = train_test_split(
    temp_images,
    test_size=(test_split / (test_split + val_split)),
    stratify=[label for _, label in temp_images],
    random_state=42
)

# Copy files into respective directories without preprocessing
def copy_images(images, target_dir):
    for img_path, label in images:
        label_dir = os.path.join(target_dir, label)
        os.makedirs(label_dir, exist_ok=True)
        shutil.copy(img_path, label_dir)

copy_images(train_images, train_dir)
copy_images(val_images, val_dir)
copy_images(test_images, test_dir)

# Parameters
input_shape = (128, 128, 3)
batch_size = 32
epochs = 20

# Create data generators for binary classification
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='binary'
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

# Print the number of samples in each split
print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {validation_generator.samples}")
print(f"Test samples: {test_generator.samples}")

# Compute class weights
train_labels = train_generator.classes
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)
class_weights = dict(enumerate(class_weights))

# Print the calculated class weights
print("Class Weights:", class_weights)

# Load pre-trained VGG19 model + higher-level layers
base_model = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze the base model
base_model.trainable = False

# Add Global Average Pooling and output layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
output = Dense(1, activation='sigmoid')(x)  # Binary classification output

# Create the complete model
model_vgg = Model(inputs=base_model.input, outputs=output)

# Compile the model
model_vgg.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Start profiling for training
start_time = time.time()
history_vgg = model_vgg.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    class_weight=class_weights
)
print("Training time: {:.2f} seconds".format(time.time() - start_time))

# Evaluate the model on the test set
start_time = time.time()
loss, accuracy = model_vgg.evaluate(test_generator)
print("Test evaluation time: {:.2f} seconds".format(time.time() - start_time))

Found 1273 images belonging to 2 classes.
Found 182 images belonging to 2 classes.
Found 365 images belonging to 2 classes.
Training samples: 1273
Validation samples: 182
Test samples: 365
Class Weights: {0: 1.548661800486618, 1: 0.738399071925754}
Epoch 1/20


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


40/40 ━━━━━━━━━━━━━━━━━━━━ 10s 179ms/step - accuracy: 0.3386 - loss: 0.7143 - val_accuracy: 0.3791 - val_loss: 0.7274
Epoch 2/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 146ms/step - accuracy: 0.3770 - loss: 0.6950 - val_accuracy: 0.4560 - val_loss: 0.6996
Epoch 3/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 145ms/step - accuracy: 0.4276 - loss: 0.6946 - val_accuracy: 0.5220 - val_loss: 0.6890
Epoch 4/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 154ms/step - accuracy: 0.5136 - loss: 0.6800 - val_accuracy: 0.5824 - val_loss: 0.6830
Epoch 5/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 151ms/step - accuracy: 0.5265 - loss: 0.6822 - val_accuracy: 0.5604 - val_loss: 0.6831
Epoch 6/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 156ms/step - accuracy: 0.5211 - loss: 0.6774 - val_accuracy: 0.5934 - val_loss: 0.6790
Epoch 7/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 155ms/step - accuracy: 0.5557 - loss: 0.6756 - val_accuracy: 0.5659 - val_loss: 0.6790
Epoch 8/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 153ms/step - accuracy: 0.5476 - loss: 0.6825 - val_accuracy: 0.5714 - val

# BUSI

In [17]:
print("Welcome")

Welcome


# Final DNBCD Model (Custom CNN + DenseNet + Transfer Learning)

In [18]:
import os 
import numpy as np
import cv2  # OpenCV for general use (not CLAHE)
import tensorflow as tf
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import DenseNet121
from sklearn.utils.class_weight import compute_class_weight
import time

# Define paths
dataset_path = '/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT'
base_dir = '/kaggle/working/split_data'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

# Ensure clean split directories
for dir_path in [train_dir, val_dir, test_dir]:
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
    os.makedirs(dir_path)

# Split dataset into 70% train, 20% test, 10% validation
all_images = []
for class_name in os.listdir(dataset_path):
    class_dir = os.path.join(dataset_path, class_name)
    images = [os.path.join(class_dir, img) for img in os.listdir(class_dir)]
    all_images.extend([(img, class_name) for img in images])

test_split = 0.2
val_split = 0.1
train_images, temp_images = train_test_split(
    all_images,
    test_size=(test_split + val_split),
    stratify=[label for _, label in all_images],
    random_state=42
)
val_images, test_images = train_test_split(
    temp_images,
    test_size=(test_split / (test_split + val_split)),
    stratify=[label for _, label in temp_images],
    random_state=42
)

# Copy files into respective directories without preprocessing
def copy_images(images, target_dir):
    for img_path, label in images:
        label_dir = os.path.join(target_dir, label)
        os.makedirs(label_dir, exist_ok=True)
        shutil.copy(img_path, label_dir)

copy_images(train_images, train_dir)
copy_images(val_images, val_dir)
copy_images(test_images, test_dir)

# Parameters
input_shape = (128, 128, 3)
batch_size = 32
epochs = 20

# Create data generators for binary classification
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'  # Categorical classification
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'  # Categorical classification
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',  # Categorical classification
    shuffle=False
)

# Print the number of samples in each split
print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {validation_generator.samples}")
print(f"Test samples: {test_generator.samples}")

# Compute class weights
train_labels = train_generator.classes
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)
class_weights = dict(enumerate(class_weights))

# Print the calculated class weights
print("Class Weights:", class_weights)

# Load pre-trained DenseNet121 model + higher-level layers
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze the base model
base_model.trainable = False

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(3, activation='softmax')(x)  # Categorical classification

# Create the complete model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Start profiling for initial training
start_time = time.time()
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    class_weight=class_weights
)
print("Initial training time: {:.2f} seconds".format(time.time() - start_time))

# Unfreeze the base model for fine-tuning
base_model.trainable = True
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])  # Reduce learning rate for fine-tuning

# Start profiling for fine-tuning
start_time = time.time()
history_fine = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    class_weight=class_weights
)
print("Fine-tuning training time: {:.2f} seconds".format(time.time() - start_time))

# Evaluate the model on the test set
start_time = time.time()
loss, accuracy = model.evaluate(test_generator)
print("Test evaluation time: {:.2f} seconds".format(time.time() - start_time))

Found 1104 images belonging to 3 classes.
Found 158 images belonging to 3 classes.
Found 316 images belonging to 3 classes.
Training samples: 1104
Validation samples: 158
Test samples: 316
Class Weights: {0: 0.5906902086677368, 1: 1.2474576271186442, 2: 1.978494623655914}
Epoch 1/20


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


35/35 ━━━━━━━━━━━━━━━━━━━━ 57s 901ms/step - accuracy: 0.5155 - loss: 1.3867 - val_accuracy: 0.7278 - val_loss: 0.5754
Epoch 2/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 216ms/step - accuracy: 0.6981 - loss: 0.6297 - val_accuracy: 0.7722 - val_loss: 0.4591
Epoch 3/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 219ms/step - accuracy: 0.6795 - loss: 0.6594 - val_accuracy: 0.7785 - val_loss: 0.4500
Epoch 4/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 214ms/step - accuracy: 0.7620 - loss: 0.5821 - val_accuracy: 0.7848 - val_loss: 0.4537
Epoch 5/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 218ms/step - accuracy: 0.7395 - loss: 0.5708 - val_accuracy: 0.8228 - val_loss: 0.4159
Epoch 6/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 217ms/step - accuracy: 0.7647 - loss: 0.4878 - val_accuracy: 0.8101 - val_loss: 0.4073
Epoch 7/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 218ms/step - accuracy: 0.7634 - loss: 0.5157 - val_accuracy: 0.8228 - val_loss: 0.3852
Epoch 8/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 10s 224ms/step - accuracy: 0.7602 - loss: 0.4965 - val_accuracy: 0.8228 - va

I0000 00:00:1740377563.254425     114 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'input_reduce_fusion_1299', 4 bytes spill stores, 4 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_reduce_fusion_1294', 8 bytes spill stores, 8 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_reduce_select_fusion_54', 4 bytes spill stores, 4 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_reduce_select_fusion_47', 4 bytes spill stores, 4 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_reduce_fusion_1040', 4 bytes spill stores, 4 bytes spill loads



19/35 ━━━━━━━━━━━━━━━━━━━━ 3s 232ms/step - accuracy: 0.7539 - loss: 0.6939

I0000 00:00:1740377636.618206     114 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'input_reduce_select_fusion_22', 4 bytes spill stores, 4 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_reduce_select_fusion_19', 4 bytes spill stores, 4 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_reduce_fusion_468', 4 bytes spill stores, 4 bytes spill loads



35/35 ━━━━━━━━━━━━━━━━━━━━ 206s 3s/step - accuracy: 0.7620 - loss: 0.6308 - val_accuracy: 0.8038 - val_loss: 0.6066
Epoch 2/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 11s 249ms/step - accuracy: 0.8013 - loss: 0.3868 - val_accuracy: 0.7089 - val_loss: 0.7301
Epoch 3/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 10s 245ms/step - accuracy: 0.8458 - loss: 0.3540 - val_accuracy: 0.7152 - val_loss: 0.6120
Epoch 4/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 11s 245ms/step - accuracy: 0.8500 - loss: 0.3476 - val_accuracy: 0.7532 - val_loss: 0.4939
Epoch 5/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 10s 244ms/step - accuracy: 0.8651 - loss: 0.2732 - val_accuracy: 0.7785 - val_loss: 0.4518
Epoch 6/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 10s 234ms/step - accuracy: 0.8858 - loss: 0.2727 - val_accuracy: 0.8861 - val_loss: 0.3559
Epoch 7/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 10s 244ms/step - accuracy: 0.8894 - loss: 0.2888 - val_accuracy: 0.8734 - val_loss: 0.2551
Epoch 8/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 10s 244ms/step - accuracy: 0.9078 - loss: 0.1887 - val_accuracy: 0.9367 

# Mobilenet+transfer

In [19]:
import os
import numpy as np
import cv2  # OpenCV for general use (not CLAHE)
import tensorflow as tf
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNet
from sklearn.utils.class_weight import compute_class_weight
import time

# Define paths
dataset_path = '/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT'
base_dir = '/kaggle/working/split_data'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

# Ensure clean split directories
for dir_path in [train_dir, val_dir, test_dir]:
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
    os.makedirs(dir_path)

# Split dataset into 70% train, 20% test, 10% validation
all_images = []
for class_name in os.listdir(dataset_path):
    class_dir = os.path.join(dataset_path, class_name)
    images = [os.path.join(class_dir, img) for img in os.listdir(class_dir)]
    all_images.extend([(img, class_name) for img in images])

test_split = 0.2
val_split = 0.1
train_images, temp_images = train_test_split(
    all_images,
    test_size=(test_split + val_split),
    stratify=[label for _, label in all_images],
    random_state=42
)
val_images, test_images = train_test_split(
    temp_images,
    test_size=(test_split / (test_split + val_split)),
    stratify=[label for _, label in temp_images],
    random_state=42
)

# Copy files into respective directories without preprocessing
def copy_images(images, target_dir):
    for img_path, label in images:
        label_dir = os.path.join(target_dir, label)
        os.makedirs(label_dir, exist_ok=True)
        shutil.copy(img_path, label_dir)  # Copy the original image without preprocessing

copy_images(train_images, train_dir)
copy_images(val_images, val_dir)
copy_images(test_images, test_dir)

# Parameters
input_shape = (128, 128, 3)
batch_size = 32
epochs = 20

# Create data generators for classification
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'  # Categorical classification
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'  # Categorical classification
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',  # Categorical classification
    shuffle=False
)

# Print the number of samples in each split
print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {validation_generator.samples}")
print(f"Test samples: {test_generator.samples}")

# Compute class weights
train_labels = train_generator.classes  # Class indices for training samples
class_weights_array = compute_class_weight(
    class_weight='balanced',  # Balance classes based on their frequency
    classes=np.unique(train_labels),
    y=train_labels
)
class_weights = dict(enumerate(class_weights_array))

# Print the calculated class weights
print("Class Weights:", class_weights)

# Load pre-trained MobileNet model + higher-level layers
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze the base model
base_model.trainable = False

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(3, activation='softmax')(x)  # Categorical classification

# Create the complete model
mobilenet_model = Model(inputs=base_model.input, outputs=output)

# Compile the model
mobilenet_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Start profiling for initial training
start_time = time.time()
mobilenet_history_initial = mobilenet_model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    class_weight=class_weights  # Use calculated class weights
)
print("Initial training time: {:.2f} seconds".format(time.time() - start_time))

# Unfreeze the base model for fine-tuning
base_model.trainable = True
mobilenet_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])  # Reduce learning rate for fine-tuning

# Start profiling for fine-tuning
start_time = time.time()
mobilenet_history_fine = mobilenet_model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    class_weight=class_weights
)
print("Fine-tuning training time: {:.2f} seconds".format(time.time() - start_time))

# Evaluate the model on the test set
start_time = time.time()
loss, accuracy = mobilenet_model.evaluate(test_generator)
print("Test evaluation time: {:.2f} seconds".format(time.time() - start_time))

Found 1104 images belonging to 3 classes.
Found 158 images belonging to 3 classes.
Found 316 images belonging to 3 classes.
Training samples: 1104
Validation samples: 158
Test samples: 316
Class Weights: {0: 0.5906902086677368, 1: 1.2474576271186442, 2: 1.978494623655914}
Epoch 1/20


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


35/35 ━━━━━━━━━━━━━━━━━━━━ 22s 413ms/step - accuracy: 0.5353 - loss: 1.2023 - val_accuracy: 0.6962 - val_loss: 0.5678
Epoch 2/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 205ms/step - accuracy: 0.7333 - loss: 0.6353 - val_accuracy: 0.7722 - val_loss: 0.4549
Epoch 3/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 207ms/step - accuracy: 0.7503 - loss: 0.5380 - val_accuracy: 0.8354 - val_loss: 0.3962
Epoch 4/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 204ms/step - accuracy: 0.7888 - loss: 0.4761 - val_accuracy: 0.8608 - val_loss: 0.3662
Epoch 5/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 205ms/step - accuracy: 0.7823 - loss: 0.5259 - val_accuracy: 0.8165 - val_loss: 0.3527
Epoch 6/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 205ms/step - accuracy: 0.8094 - loss: 0.4077 - val_accuracy: 0.7911 - val_loss: 0.3890
Epoch 7/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 210ms/step - accuracy: 0.8083 - loss: 0.4422 - val_accuracy: 0.8165 - val_loss: 0.3722
Epoch 8/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 206ms/step - accuracy: 0.8058 - loss: 0.4462 - val_accuracy: 0.8101 - val

# Resnet50+transfer

In [20]:
import os
import numpy as np
import cv2  # OpenCV for general use (not CLAHE)
import tensorflow as tf
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from sklearn.utils.class_weight import compute_class_weight
import time

# Optional: For reproducibility
import random
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

# Define paths
dataset_path = '/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT'
base_dir = '/kaggle/working/split_data'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

# Ensure clean split directories
for dir_path in [train_dir, val_dir, test_dir]:
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
    os.makedirs(dir_path)

# Split dataset into 70% train, 20% test, 10% validation
all_images = []
for class_name in os.listdir(dataset_path):
    class_dir = os.path.join(dataset_path, class_name)
    images = [os.path.join(class_dir, img) for img in os.listdir(class_dir)]
    all_images.extend([(img, class_name) for img in images])

test_split = 0.2
val_split = 0.1
train_images, temp_images = train_test_split(
    all_images,
    test_size=(test_split + val_split),
    stratify=[label for _, label in all_images],
    random_state=42
)
val_images, test_images = train_test_split(
    temp_images,
    test_size=(test_split / (test_split + val_split)),
    stratify=[label for _, label in temp_images],
    random_state=42
)

# Copy files into respective directories without preprocessing
def copy_images(images, target_dir):
    for img_path, label in images:
        label_dir = os.path.join(target_dir, label)
        os.makedirs(label_dir, exist_ok=True)
        shutil.copy(img_path, label_dir)  # Copy the original image without preprocessing

copy_images(train_images, train_dir)
copy_images(val_images, val_dir)
copy_images(test_images, test_dir)

# Parameters
input_shape = (128, 128, 3)
batch_size = 32
epochs = 20

# Create data generators for classification
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'  # Categorical classification
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'  # Categorical classification
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',  # Categorical classification
    shuffle=False
)

# Print the number of samples in each split
print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {validation_generator.samples}")
print(f"Test samples: {test_generator.samples}")

# Compute class weights
train_labels = train_generator.classes  # Class indices for training samples
class_weights_array = compute_class_weight(
    class_weight='balanced',  # Balance classes based on their frequency
    classes=np.unique(train_labels),
    y=train_labels
)
class_weights = dict(enumerate(class_weights_array))

# Print the calculated class weights
print("Class Weights:", class_weights)

# Load pre-trained ResNet50 model + higher-level layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze the base model
base_model.trainable = False

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(3, activation='softmax')(x)  # Categorical classification

# Create the complete model
resnet50_model = Model(inputs=base_model.input, outputs=output)

# Compile the model
resnet50_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Start profiling for initial training
start_time = time.time()
resnet50_history_initial = resnet50_model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    class_weight=class_weights  # Use calculated class weights
)
print("Initial training time: {:.2f} seconds".format(time.time() - start_time))

# Unfreeze the base model for fine-tuning
base_model.trainable = True
resnet50_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])  # Reduce learning rate for fine-tuning

# Start profiling for fine-tuning
start_time = time.time()
resnet50_history_fine = resnet50_model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    class_weight=class_weights
)
print("Fine-tuning training time: {:.2f} seconds".format(time.time() - start_time))

# Evaluate the model on the test set
start_time = time.time()
loss, accuracy = resnet50_model.evaluate(test_generator)
print("Test evaluation time: {:.2f} seconds".format(time.time() - start_time))

Found 1104 images belonging to 3 classes.
Found 158 images belonging to 3 classes.
Found 316 images belonging to 3 classes.
Training samples: 1104
Validation samples: 158
Test samples: 316
Class Weights: {0: 0.5906902086677368, 1: 1.2474576271186442, 2: 1.978494623655914}
Epoch 1/20


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


35/35 ━━━━━━━━━━━━━━━━━━━━ 38s 551ms/step - accuracy: 0.3370 - loss: 1.2480 - val_accuracy: 0.3608 - val_loss: 1.0439
Epoch 2/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 211ms/step - accuracy: 0.3908 - loss: 1.0800 - val_accuracy: 0.3291 - val_loss: 1.0349
Epoch 3/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 208ms/step - accuracy: 0.3666 - loss: 1.0639 - val_accuracy: 0.4114 - val_loss: 0.9722
Epoch 4/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 213ms/step - accuracy: 0.3786 - loss: 1.0293 - val_accuracy: 0.5823 - val_loss: 0.9395
Epoch 5/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 217ms/step - accuracy: 0.3873 - loss: 1.0000 - val_accuracy: 0.3354 - val_loss: 1.0073
Epoch 6/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 209ms/step - accuracy: 0.3774 - loss: 1.0139 - val_accuracy: 0.4810 - val_loss: 0.9301
Epoch 7/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 208ms/step - accuracy: 0.4490 - loss: 0.9822 - val_accuracy: 0.3671 - val_loss: 0.9025
Epoch 8/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 212ms/step - accuracy: 0.4295 - loss: 0.9881 - val_accuracy: 0.5063 - val

I0000 00:00:1740378529.782014     116 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_26', 16 bytes spill stores, 16 bytes spill loads



35/35 ━━━━━━━━━━━━━━━━━━━━ 82s 956ms/step - accuracy: 0.6725 - loss: 9.5888 - val_accuracy: 0.2848 - val_loss: 36.3561
Epoch 2/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 220ms/step - accuracy: 0.6981 - loss: 1.6946 - val_accuracy: 0.3101 - val_loss: 15.1564
Epoch 3/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 220ms/step - accuracy: 0.7844 - loss: 0.5854 - val_accuracy: 0.2975 - val_loss: 10.0877
Epoch 4/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 10s 225ms/step - accuracy: 0.7953 - loss: 0.5176 - val_accuracy: 0.2848 - val_loss: 5.0473
Epoch 5/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 221ms/step - accuracy: 0.8029 - loss: 0.4163 - val_accuracy: 0.2911 - val_loss: 7.4691
Epoch 6/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 219ms/step - accuracy: 0.8374 - loss: 0.3795 - val_accuracy: 0.2975 - val_loss: 4.1320
Epoch 7/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 221ms/step - accuracy: 0.8365 - loss: 0.3924 - val_accuracy: 0.4810 - val_loss: 1.0503
Epoch 8/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 10s 220ms/step - accuracy: 0.8465 - loss: 0.4152 - val_accuracy: 0.5823 

# Vgg19+transfer

In [21]:
import os
import numpy as np
import cv2  # OpenCV for general use (not CLAHE)
import tensorflow as tf
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from sklearn.utils.class_weight import compute_class_weight
import time

# Optional: For reproducibility
import random
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

# Define paths
dataset_path = '/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT'
base_dir = '/kaggle/working/split_data'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

# Ensure clean split directories
for dir_path in [train_dir, val_dir, test_dir]:
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
    os.makedirs(dir_path)

# Split dataset into 70% train, 20% test, 10% validation
all_images = []
for class_name in os.listdir(dataset_path):
    class_dir = os.path.join(dataset_path, class_name)
    images = [os.path.join(class_dir, img) for img in os.listdir(class_dir)]
    all_images.extend([(img, class_name) for img in images])

test_split = 0.2
val_split = 0.1
train_images, temp_images = train_test_split(
    all_images,
    test_size=(test_split + val_split),
    stratify=[label for _, label in all_images],
    random_state=42
)
val_images, test_images = train_test_split(
    temp_images,
    test_size=(test_split / (test_split + val_split)),
    stratify=[label for _, label in temp_images],
    random_state=42
)

# Copy files into respective directories without preprocessing
def copy_images(images, target_dir):
    for img_path, label in images:
        label_dir = os.path.join(target_dir, label)
        os.makedirs(label_dir, exist_ok=True)
        shutil.copy(img_path, label_dir)  # Copy the original image without preprocessing

copy_images(train_images, train_dir)
copy_images(val_images, val_dir)
copy_images(test_images, test_dir)

# Parameters
input_shape = (128, 128, 3)
batch_size = 32
epochs = 20

# Create data generators for classification
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # VGG19-specific preprocessing
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'  # Categorical classification
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'  # Categorical classification
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',  # Categorical classification
    shuffle=False
)

# Print the number of samples in each split
print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {validation_generator.samples}")
print(f"Test samples: {test_generator.samples}")

# Compute class weights
train_labels = train_generator.classes  # Class indices for training samples
class_weights_array = compute_class_weight(
    class_weight='balanced',  # Balance classes based on their frequency
    classes=np.unique(train_labels),
    y=train_labels
)
class_weights = dict(enumerate(class_weights_array))

# Print the calculated class weights
print("Class Weights:", class_weights)

# Load pre-trained VGG19 model + higher-level layers
base_model = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze the base model
base_model.trainable = False

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(3, activation='softmax')(x)  # Categorical classification

# Create the complete model
vgg19_model = Model(inputs=base_model.input, outputs=output)

# Compile the model
vgg19_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Start profiling for initial training
start_time = time.time()
vgg19_history_initial = vgg19_model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    class_weight=class_weights  # Use calculated class weights
)
print("Initial training time: {:.2f} seconds".format(time.time() - start_time))

# Unfreeze the base model for fine-tuning
base_model.trainable = True
vgg19_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])  # Reduce learning rate for fine-tuning

# Start profiling for fine-tuning
start_time = time.time()
vgg19_history_fine = vgg19_model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    class_weight=class_weights
)
print("Fine-tuning training time: {:.2f} seconds".format(time.time() - start_time))

# Evaluate the model on the test set
start_time = time.time()
loss, accuracy = vgg19_model.evaluate(test_generator)
print("Test evaluation time: {:.2f} seconds".format(time.time() - start_time))

Found 1104 images belonging to 3 classes.
Found 158 images belonging to 3 classes.
Found 316 images belonging to 3 classes.
Training samples: 1104
Validation samples: 158
Test samples: 316
Class Weights: {0: 0.5906902086677368, 1: 1.2474576271186442, 2: 1.978494623655914}
Epoch 1/20


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


35/35 ━━━━━━━━━━━━━━━━━━━━ 22s 462ms/step - accuracy: 0.5326 - loss: 3.0884 - val_accuracy: 0.6962 - val_loss: 0.6144
Epoch 2/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 218ms/step - accuracy: 0.6566 - loss: 1.0949 - val_accuracy: 0.7405 - val_loss: 0.4611
Epoch 3/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 215ms/step - accuracy: 0.6878 - loss: 0.8228 - val_accuracy: 0.8544 - val_loss: 0.3948
Epoch 4/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 210ms/step - accuracy: 0.7125 - loss: 0.6798 - val_accuracy: 0.8608 - val_loss: 0.3954
Epoch 5/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 215ms/step - accuracy: 0.7468 - loss: 0.5527 - val_accuracy: 0.7278 - val_loss: 0.4547
Epoch 6/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 210ms/step - accuracy: 0.7001 - loss: 0.6938 - val_accuracy: 0.8291 - val_loss: 0.4074
Epoch 7/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 216ms/step - accuracy: 0.7760 - loss: 0.5131 - val_accuracy: 0.8544 - val_loss: 0.4035
Epoch 8/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 212ms/step - accuracy: 0.7752 - loss: 0.5089 - val_accuracy: 0.8038 - val

# DenseNet121

In [22]:
import os
import numpy as np
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import DenseNet121
from sklearn.utils.class_weight import compute_class_weight
import time

# Define paths
dataset_path = '/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT'
base_dir = '/kaggle/working/split_data'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

# Ensure clean split directories
for dir_path in [train_dir, val_dir, test_dir]:
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
    os.makedirs(dir_path)

# Split dataset into 70% train, 20% test, 10% validation
all_images = []
for class_name in os.listdir(dataset_path):
    class_dir = os.path.join(dataset_path, class_name)
    images = [os.path.join(class_dir, img) for img in os.listdir(class_dir)]
    all_images.extend([(img, class_name) for img in images])

test_split = 0.2
val_split = 0.1
train_images, temp_images = train_test_split(
    all_images,
    test_size=(test_split + val_split),
    stratify=[label for _, label in all_images],
    random_state=42
)
val_images, test_images = train_test_split(
    temp_images,
    test_size=(test_split / (test_split + val_split)),
    stratify=[label for _, label in temp_images],
    random_state=42
)

# Copy files into respective directories without preprocessing
def copy_images(images, target_dir):
    for img_path, label in images:
        label_dir = os.path.join(target_dir, label)
        os.makedirs(label_dir, exist_ok=True)
        shutil.copy(img_path, label_dir)  # Copy the original image without preprocessing

copy_images(train_images, train_dir)
copy_images(val_images, val_dir)
copy_images(test_images, test_dir)

# Parameters
input_shape = (128, 128, 3)
batch_size = 32
epochs = 20

# Create data generators for classification
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'  # Categorical classification
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'  # Categorical classification
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',  # Categorical classification
    shuffle=False
)

# Print the number of samples in each split
print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {validation_generator.samples}")
print(f"Test samples: {test_generator.samples}")

# Compute class weights
train_labels = train_generator.classes  # Class indices for training samples
class_weights = compute_class_weight(
    class_weight='balanced',  # Balance classes based on their frequency
    classes=np.unique(train_labels),
    y=train_labels
)
class_weights = dict(enumerate(class_weights))

# Print the calculated class weights
print("Class Weights:", class_weights)

# Load pre-trained DenseNet121 model + higher-level layers
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze the base model
base_model.trainable = False

# Add Global Average Pooling and output layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
output = Dense(3, activation='softmax')(x)  # Categorical classification output

# Create the complete model
model_densenet = Model(inputs=base_model.input, outputs=output)

# Compile the model
model_densenet.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Start profiling for training
start_time = time.time()
history_densenet = model_densenet.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    class_weight=class_weights
)
print("Training time: {:.2f} seconds".format(time.time() - start_time))

# Evaluate the model on the test set
start_time = time.time()
loss, accuracy = model_densenet.evaluate(test_generator)
print("Test evaluation time: {:.2f} seconds".format(time.time() - start_time))

Found 1104 images belonging to 3 classes.
Found 158 images belonging to 3 classes.
Found 316 images belonging to 3 classes.
Training samples: 1104
Validation samples: 158
Test samples: 316
Class Weights: {0: 0.5906902086677368, 1: 1.2474576271186442, 2: 1.978494623655914}
Epoch 1/20


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


35/35 ━━━━━━━━━━━━━━━━━━━━ 42s 695ms/step - accuracy: 0.2897 - loss: 1.2672 - val_accuracy: 0.5127 - val_loss: 0.9783
Epoch 2/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 212ms/step - accuracy: 0.4412 - loss: 1.1534 - val_accuracy: 0.6139 - val_loss: 0.8844
Epoch 3/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 211ms/step - accuracy: 0.5335 - loss: 0.9889 - val_accuracy: 0.6329 - val_loss: 0.8110
Epoch 4/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 217ms/step - accuracy: 0.5891 - loss: 0.9017 - val_accuracy: 0.6646 - val_loss: 0.7617
Epoch 5/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 217ms/step - accuracy: 0.6346 - loss: 0.8405 - val_accuracy: 0.6899 - val_loss: 0.7184
Epoch 6/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 219ms/step - accuracy: 0.6528 - loss: 0.8065 - val_accuracy: 0.6962 - val_loss: 0.6863
Epoch 7/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 212ms/step - accuracy: 0.6587 - loss: 0.8208 - val_accuracy: 0.7215 - val_loss: 0.6557
Epoch 8/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 220ms/step - accuracy: 0.6799 - loss: 0.7354 - val_accuracy: 0.7215 - val

# Mobilenet

In [23]:
import os
import numpy as np
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNet
from sklearn.utils.class_weight import compute_class_weight
import time

# Define paths
dataset_path = '/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT'
base_dir = '/kaggle/working/split_data'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

# Ensure clean split directories
for dir_path in [train_dir, val_dir, test_dir]:
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
    os.makedirs(dir_path)

# Split dataset into 70% train, 20% test, 10% validation
all_images = []
for class_name in os.listdir(dataset_path):
    class_dir = os.path.join(dataset_path, class_name)
    images = [os.path.join(class_dir, img) for img in os.listdir(class_dir)]
    all_images.extend([(img, class_name) for img in images])

test_split = 0.2
val_split = 0.1
train_images, temp_images = train_test_split(
    all_images,
    test_size=(test_split + val_split),
    stratify=[label for _, label in all_images],
    random_state=42
)
val_images, test_images = train_test_split(
    temp_images,
    test_size=(test_split / (test_split + val_split)),
    stratify=[label for _, label in temp_images],
    random_state=42
)

# Copy files into respective directories without preprocessing
def copy_images(images, target_dir):
    for img_path, label in images:
        label_dir = os.path.join(target_dir, label)
        os.makedirs(label_dir, exist_ok=True)
        shutil.copy(img_path, label_dir)  # Copy the original image without preprocessing

copy_images(train_images, train_dir)
copy_images(val_images, val_dir)
copy_images(test_images, test_dir)

# Parameters
input_shape = (128, 128, 3)
batch_size = 32
epochs = 20

# Create data generators for classification
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'  # Categorical classification
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'  # Categorical classification
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',  # Categorical classification
    shuffle=False
)

# Print the number of samples in each split
print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {validation_generator.samples}")
print(f"Test samples: {test_generator.samples}")

# Compute class weights
train_labels = train_generator.classes  # Class indices for training samples
class_weights = compute_class_weight(
    class_weight='balanced',  # Balance classes based on their frequency
    classes=np.unique(train_labels),
    y=train_labels
)
class_weights = dict(enumerate(class_weights))

# Print the calculated class weights
print("Class Weights:", class_weights)

# Load pre-trained MobileNet model + higher-level layers
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze the base model
base_model.trainable = False

# Add Global Average Pooling and output layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
output = Dense(3, activation='softmax')(x)  # Categorical classification output

# Create the complete model
model_mobilenet = Model(inputs=base_model.input, outputs=output)

# Compile the model
model_mobilenet.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Start profiling for training
start_time = time.time()
history_mobilenet = model_mobilenet.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    class_weight=class_weights
)
print("Training time: {:.2f} seconds".format(time.time() - start_time))

# Evaluate the model on the test set
start_time = time.time()
loss, accuracy = model_mobilenet.evaluate(test_generator)
print("Test evaluation time: {:.2f} seconds".format(time.time() - start_time))

Found 1104 images belonging to 3 classes.
Found 158 images belonging to 3 classes.
Found 316 images belonging to 3 classes.
Training samples: 1104
Validation samples: 158
Test samples: 316
Class Weights: {0: 0.5906902086677368, 1: 1.2474576271186442, 2: 1.978494623655914}
Epoch 1/20


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


35/35 ━━━━━━━━━━━━━━━━━━━━ 15s 286ms/step - accuracy: 0.5167 - loss: 1.1503 - val_accuracy: 0.4873 - val_loss: 1.0009
Epoch 2/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 207ms/step - accuracy: 0.5233 - loss: 1.0309 - val_accuracy: 0.5063 - val_loss: 0.9025
Epoch 3/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 210ms/step - accuracy: 0.5783 - loss: 0.9220 - val_accuracy: 0.5633 - val_loss: 0.8115
Epoch 4/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 209ms/step - accuracy: 0.6135 - loss: 0.8698 - val_accuracy: 0.6076 - val_loss: 0.7585
Epoch 5/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 213ms/step - accuracy: 0.6412 - loss: 0.8156 - val_accuracy: 0.6835 - val_loss: 0.7256
Epoch 6/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 209ms/step - accuracy: 0.7033 - loss: 0.7620 - val_accuracy: 0.7089 - val_loss: 0.6854
Epoch 7/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 207ms/step - accuracy: 0.7092 - loss: 0.7248 - val_accuracy: 0.7152 - val_loss: 0.6438
Epoch 8/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 206ms/step - accuracy: 0.7293 - loss: 0.6490 - val_accuracy: 0.7152 - val

# Resnet50

In [24]:
import os
import numpy as np
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from sklearn.utils.class_weight import compute_class_weight
import time

# Define paths
dataset_path = '/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT'
base_dir = '/kaggle/working/split_data'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

# Ensure clean split directories
for dir_path in [train_dir, val_dir, test_dir]:
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
    os.makedirs(dir_path)

# Split dataset into 70% train, 20% test, 10% validation
all_images = []
for class_name in os.listdir(dataset_path):
    class_dir = os.path.join(dataset_path, class_name)
    images = [os.path.join(class_dir, img) for img in os.listdir(class_dir)]
    all_images.extend([(img, class_name) for img in images])

test_split = 0.2
val_split = 0.1
train_images, temp_images = train_test_split(
    all_images,
    test_size=(test_split + val_split),
    stratify=[label for _, label in all_images],
    random_state=42
)
val_images, test_images = train_test_split(
    temp_images,
    test_size=(test_split / (test_split + val_split)),
    stratify=[label for _, label in temp_images],
    random_state=42
)

# Copy files into respective directories without preprocessing
def copy_images(images, target_dir):
    for img_path, label in images:
        label_dir = os.path.join(target_dir, label)
        os.makedirs(label_dir, exist_ok=True)
        shutil.copy(img_path, label_dir)  # Copy the original image without preprocessing

copy_images(train_images, train_dir)
copy_images(val_images, val_dir)
copy_images(test_images, test_dir)

# Parameters
input_shape = (128, 128, 3)
batch_size = 32
epochs = 20

# Create data generators for classification
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'  # Categorical classification
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'  # Categorical classification
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',  # Categorical classification
    shuffle=False
)

# Print the number of samples in each split
print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {validation_generator.samples}")
print(f"Test samples: {test_generator.samples}")

# Compute class weights
train_labels = train_generator.classes  # Class indices for training samples
class_weights = compute_class_weight(
    class_weight='balanced',  # Balance classes based on their frequency
    classes=np.unique(train_labels),
    y=train_labels
)
class_weights = dict(enumerate(class_weights))

# Print the calculated class weights
print("Class Weights:", class_weights)

# Load pre-trained ResNet50 model + higher-level layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze the base model
base_model.trainable = False

# Add Global Average Pooling and output layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
output = Dense(3, activation='softmax')(x)  # Categorical classification output

# Create the complete model
model_resnet = Model(inputs=base_model.input, outputs=output)

# Compile the model
model_resnet.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Start profiling for training
start_time = time.time()
history_resnet = model_resnet.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    class_weight=class_weights
)
print("Training time: {:.2f} seconds".format(time.time() - start_time))

# Evaluate the model on the test set
start_time = time.time()
loss, accuracy = model_resnet.evaluate(test_generator)
print("Test evaluation time: {:.2f} seconds".format(time.time() - start_time))

Found 1104 images belonging to 3 classes.
Found 158 images belonging to 3 classes.
Found 316 images belonging to 3 classes.
Training samples: 1104
Validation samples: 158
Test samples: 316
Class Weights: {0: 0.5906902086677368, 1: 1.2474576271186442, 2: 1.978494623655914}
Epoch 1/20


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


35/35 ━━━━━━━━━━━━━━━━━━━━ 23s 382ms/step - accuracy: 0.2697 - loss: 1.4631 - val_accuracy: 0.2658 - val_loss: 1.6134
Epoch 2/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 217ms/step - accuracy: 0.2747 - loss: 1.3067 - val_accuracy: 0.2658 - val_loss: 1.4608
Epoch 3/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 214ms/step - accuracy: 0.2646 - loss: 1.2299 - val_accuracy: 0.2658 - val_loss: 1.3431
Epoch 4/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 212ms/step - accuracy: 0.2672 - loss: 1.1667 - val_accuracy: 0.2658 - val_loss: 1.2544
Epoch 5/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 211ms/step - accuracy: 0.2608 - loss: 1.1323 - val_accuracy: 0.2595 - val_loss: 1.1870
Epoch 6/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 210ms/step - accuracy: 0.2635 - loss: 1.1075 - val_accuracy: 0.2658 - val_loss: 1.1426
Epoch 7/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 216ms/step - accuracy: 0.2741 - loss: 1.0955 - val_accuracy: 0.3228 - val_loss: 1.1133
Epoch 8/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 209ms/step - accuracy: 0.2520 - loss: 1.0874 - val_accuracy: 0.2342 - val

# Vgg19

In [25]:
import os
import numpy as np
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG19
from sklearn.utils.class_weight import compute_class_weight
import time

# Define paths
dataset_path = '/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT'
base_dir = '/kaggle/working/split_data'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

# Ensure clean split directories
for dir_path in [train_dir, val_dir, test_dir]:
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
    os.makedirs(dir_path)

# Split dataset into 70% train, 20% test, 10% validation
all_images = []
for class_name in os.listdir(dataset_path):
    class_dir = os.path.join(dataset_path, class_name)
    images = [os.path.join(class_dir, img) for img in os.listdir(class_dir)]
    all_images.extend([(img, class_name) for img in images])

test_split = 0.2
val_split = 0.1
train_images, temp_images = train_test_split(
    all_images,
    test_size=(test_split + val_split),
    stratify=[label for _, label in all_images],
    random_state=42
)
val_images, test_images = train_test_split(
    temp_images,
    test_size=(test_split / (test_split + val_split)),
    stratify=[label for _, label in temp_images],
    random_state=42
)

# Copy files into respective directories without preprocessing
def copy_images(images, target_dir):
    for img_path, label in images:
        label_dir = os.path.join(target_dir, label)
        os.makedirs(label_dir, exist_ok=True)
        shutil.copy(img_path, label_dir)  # Copy the original image without preprocessing

copy_images(train_images, train_dir)
copy_images(val_images, val_dir)
copy_images(test_images, test_dir)

# Parameters
input_shape = (128, 128, 3)
batch_size = 32
epochs = 20

# Create data generators for classification
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'  # Categorical classification
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'  # Categorical classification
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',  # Categorical classification
    shuffle=False
)

# Print the number of samples in each split
print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {validation_generator.samples}")
print(f"Test samples: {test_generator.samples}")

# Compute class weights
train_labels = train_generator.classes  # Class indices for training samples
class_weights = compute_class_weight(
    class_weight='balanced',  # Balance classes based on their frequency
    classes=np.unique(train_labels),
    y=train_labels
)
class_weights = dict(enumerate(class_weights))

# Print the calculated class weights
print("Class Weights:", class_weights)

# Load pre-trained VGG19 model + higher-level layers
base_model = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze the base model
base_model.trainable = False

# Add Global Average Pooling and output layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
output = Dense(3, activation='softmax')(x)  # Categorical classification output

# Create the complete model
model_vgg = Model(inputs=base_model.input, outputs=output)

# Compile the model
model_vgg.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Start profiling for training
start_time = time.time()
history_vgg = model_vgg.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    class_weight=class_weights
)
print("Training time: {:.2f} seconds".format(time.time() - start_time))

# Evaluate the model on the test set
start_time = time.time()
loss, accuracy = model_vgg.evaluate(test_generator)
print("Test evaluation time: {:.2f} seconds".format(time.time() - start_time))

Found 1104 images belonging to 3 classes.
Found 158 images belonging to 3 classes.
Found 316 images belonging to 3 classes.
Training samples: 1104
Validation samples: 158
Test samples: 316
Class Weights: {0: 0.5906902086677368, 1: 1.2474576271186442, 2: 1.978494623655914}
Epoch 1/20


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


35/35 ━━━━━━━━━━━━━━━━━━━━ 12s 256ms/step - accuracy: 0.5736 - loss: 1.2961 - val_accuracy: 0.5633 - val_loss: 1.0309
Epoch 2/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 214ms/step - accuracy: 0.5762 - loss: 1.2009 - val_accuracy: 0.5127 - val_loss: 1.0420
Epoch 3/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 213ms/step - accuracy: 0.5240 - loss: 1.1598 - val_accuracy: 0.4620 - val_loss: 1.0587
Epoch 4/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 213ms/step - accuracy: 0.4850 - loss: 1.1267 - val_accuracy: 0.3924 - val_loss: 1.0720
Epoch 5/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 211ms/step - accuracy: 0.5003 - loss: 1.0997 - val_accuracy: 0.3861 - val_loss: 1.0777
Epoch 6/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 215ms/step - accuracy: 0.4484 - loss: 1.1275 - val_accuracy: 0.3861 - val_loss: 1.0832
Epoch 7/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 228ms/step - accuracy: 0.4542 - loss: 1.1236 - val_accuracy: 0.4367 - val_loss: 1.0762
Epoch 8/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 220ms/step - accuracy: 0.4634 - loss: 1.0935 - val_accuracy: 0.4430 - val